In [1]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import copy
#print('copy: {}'.format(copy.__version__))



Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
scipy: 1.3.0


NameError: name 'numpy' is not defined

In [ ]:
import pandas
import collections
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
dataset = pandas.read_csv('dataset/REF_PRODUIT_TAGS_train.csv',sep='|')


def strtolist(s):
    s = s.strip("[]")
    s = s.split(',')
    return s

def get_tags(dataset):
    dic = collections.defaultdict(int)
    for x in dataset['tags']:
        new_list = strtolist(x)
        for tag in new_list:
            dic[tag] += 1
    return dic
tags = get_tags(dataset)




dataset

In [ ]:
def more_rec(dic,n):
    sorted_dic = sorted(dic.items(), key=lambda kv: kv[1],reverse=True) 
    return sorted_dic[:100], sorted_dic[100:]
tags_to_keep, tags_to_remove = more_rec(tags,100)
tags_to_keep = [i[0] for i in tags_to_keep]
tags_to_remove = [i[0] for i in tags_to_remove]

In [ ]:
def sanitize_tags(dataset,tags_to_remove, tags_to_keep):
    new_dataset = dataset.copy()
    empty = 0
    for i in range(dataset.shape[0]):
        ith_element = new_dataset["tags"][i]
#         deleted =[]
        l = strtolist(ith_element)
        l1 = list(l)
        for x in l1:
            if x in tags_to_remove:
                l.remove(x)
        if len(l) == 0:
#             deleted.append(ith_element)
#             new_dataset.drop(dataset.index[i])
#             print(new_dataset.shape[0])
            new_dataset.at[i,'tags'] = None
            empty += 1
        else:
            l = [tags_to_keep.index(x) for x in l]
            new_dataset.at[i,'tags'] = l
    
    return new_dataset.dropna(), empty#,deleted

new_dataset, empty = sanitize_tags(dataset, tags_to_remove, tags_to_keep)
new_dataset.dropna()


In [ ]:
{x for x in new_dataset["SEG_REF_CAPA_TYPE"]}

In [ ]:
new_dataset.shape[0]

In [ ]:
from collections import Counter

# dataset = pandas.read_csv('dataset/REF_PRODUIT_TAGS_train.csv',sep='|')
def have_rayon(dataset):
    rayon_type = Counter(dataset['Rayon'])
    rayon_list = list(dict(rayon_type).keys())
    products = []
    for element in dataset['Rayon']:
        temp = []
        for ele in rayon_list:
            if ele == element:
                temp.append(1)
            else:
                temp.append(0)
        products.append(temp)
    return products


def extract(dataset):    
    useful_data_info = ['Rayon', 'Famille', 'SEG_REF_CAPA_TYPE', 
                        'SANS_SUCRE', 'PORC', 'SANS_GLUTEN', 'BIO', 'HALAL', 'CASHER']

    products = [None] * len(dataset['Rayon'])
    for i in range(len(products)):
        products[i] = []

    for info in useful_data_info[:]:
        count = Counter(dataset[info])
        list_ = list(dict(count).keys())

        temp_count = 0
        for element in dataset[info]:
            temp = []
            
            for i in range(len(list_)):
                if list_[i] == element:
                    temp.append(i)
                    
            products[temp_count] = products[temp_count] + temp
            
            temp_count += 1
    
    return products


features = extract(new_dataset)
output = [x for x in new_dataset["tags"]]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

new_output = MultiLabelBinarizer().fit_transform(output)

features
